In [51]:
from pyspark.sql import SparkSession
import numpy as np # linear algebra
import pandas as pd # data processing , reading the csv file (pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
from pyspark.sql import functions

In [34]:
%matplotlib inline
warnings.filterwarnings("ignore")

spark = SparkSession.builder \
          .master("local[*]") \
          .appName("Credit Card Analysis") \
          .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

In [3]:
spark

In [4]:
trxns = spark\
        .read\
        .options(header="true",inferSchema='True')\
        .csv("/Users/abhi011097/Workspace/Dataset_Analysis/1_Credit_Card_Analysis/Dataset/Credit_card_transactions.csv")

trxns.registerTempTable("transactions")

In [5]:
trxns.printSchema()

root
 |-- index: integer (nullable = true)
 |-- City: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Card Type: string (nullable = true)
 |-- Exp Type: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Amount: integer (nullable = true)



In [6]:
type(trxns)

pyspark.sql.dataframe.DataFrame

In [7]:
trxns_pd=trxns.toPandas()

In [8]:
type(trxns_pd)

pandas.core.frame.DataFrame

#### Describe Dataframe

In [9]:
trxns.describe().show()

+-------+-----------------+----------------+--------+---------+--------+------+------------------+
|summary|            index|            City|    Date|Card Type|Exp Type|Gender|            Amount|
+-------+-----------------+----------------+--------+---------+--------+------+------------------+
|  count|            26052|           26052|   26052|    26052|   26052| 26052|             26052|
|   mean|          13025.5|            null|    null|     null|    null|  null| 156411.5374251497|
| stddev|7520.708942646298|            null|    null|     null|    null|  null|103063.25428673124|
|    min|                0| Achalpur, India|1-Apr-14|     Gold|   Bills|     F|              1005|
|    max|            26051|Zunheboto, India|9-Sep-14|   Silver|  Travel|     M|            998077|
+-------+-----------------+----------------+--------+---------+--------+------+------------------+



In [10]:
trxns_pd.describe()

,index,Amount
count,26052.000000,26052.000000
mean,13025.500000,156411.537425
std,7520.708943,103063.254287
min,0.000000,1005.000000
25%,6512.750000,77120.250000
50%,13025.500000,153106.500000
75%,19538.250000,228050.000000
max,26051.000000,998077.000000


In [11]:
trxns_pd.describe(include='O')

,City,Date,Card Type,Exp Type,Gender
count,26052,26052,26052,26052,26052
unique,986,600,4,6,2
top,"Bengaluru, India",20-Sep-14,Silver,Food,F
freq,3552,65,6840,5463,13680


#### See first 2 rows 

In [12]:
trxns.show(2)

+-----+--------------------+---------+---------+--------+------+------+
|index|                City|     Date|Card Type|Exp Type|Gender|Amount|
+-----+--------------------+---------+---------+--------+------+------+
|    0|        Delhi, India|29-Oct-14|     Gold|   Bills|     F| 82475|
|    1|Greater Mumbai, I...|22-Aug-14| Platinum|   Bills|     F| 32555|
+-----+--------------------+---------+---------+--------+------+------+
only showing top 2 rows



In [13]:
spark.sql("Select * from transactions limit 2").show()

+-----+--------------------+---------+---------+--------+------+------+
|index|                City|     Date|Card Type|Exp Type|Gender|Amount|
+-----+--------------------+---------+---------+--------+------+------+
|    0|        Delhi, India|29-Oct-14|     Gold|   Bills|     F| 82475|
|    1|Greater Mumbai, I...|22-Aug-14| Platinum|   Bills|     F| 32555|
+-----+--------------------+---------+---------+--------+------+------+



#### Find total no of records

In [14]:
trxns.count()

26052

In [15]:
spark.sql("Select count(1) from transactions").show()

+--------+
|count(1)|
+--------+
|   26052|
+--------+



#### Find min max of amount

In [16]:
print("Amount Minimum Value -> " + str(trxns.agg({'Amount': 'min'}).collect()[0][0]))
print("Amount Maximum Value -> " + str(trxns.agg({'Amount': 'max'}).collect()[0][0]))
print("Mean Value -> " + str("{:.2f}".format(trxns.agg({'Amount': 'mean'}).collect()[0][0])))
#print("Amount Maximum Value -> " + trxns.groupby('Amount').median())

Amount Minimum Value -> 1005
Amount Maximum Value -> 998077
Mean Value -> 156411.54


In [17]:
spark.sql('''Select 
            min(Amount) as min_value,  
            max(Amount) as max_value,
            round(mean(Amount),2) as mean_value
          from transactions''').show()

+---------+---------+----------+
|min_value|max_value|mean_value|
+---------+---------+----------+
|     1005|   998077| 156411.54|
+---------+---------+----------+



#### Top 5 days having most transactions

In [18]:
spark.sql('''Select 
            Date,
            count(1) as cnt
          from transactions
          Group by Date
          order by cnt desc
          limit 5
          ''').show()

+---------+---+
|     Date|cnt|
+---------+---+
|20-Sep-14| 65|
|11-Aug-14| 61|
|15-Nov-14| 61|
|21-Dec-13| 61|
|20-Feb-15| 60|
+---------+---+



In [19]:
trxns\
    .groupby('Date')\
    .count()\
    .withColumnRenamed('count','ct')\
    .sort('ct',ascending=False)\
    .limit(5)\
    .show()

+---------+---+
|     Date| ct|
+---------+---+
|20-Sep-14| 65|
|11-Aug-14| 61|
|15-Nov-14| 61|
|21-Dec-13| 61|
|20-Feb-15| 60|
+---------+---+



#### Top 5 citys having most transactions

In [20]:
spark.sql('''Select 
            city,
            count(1) as cnt
          from transactions
          Group by city
          order by cnt desc
          limit 5
          ''').show()

+--------------------+----+
|                city| cnt|
+--------------------+----+
|    Bengaluru, India|3552|
|Greater Mumbai, I...|3493|
|    Ahmedabad, India|3491|
|        Delhi, India|3482|
|    Hyderabad, India| 784|
+--------------------+----+



In [21]:
trxns\
    .groupby('city')\
    .count()\
    .sort('count',ascending=False)\
    .limit(5)\
    .show()

+--------------------+-----+
|                city|count|
+--------------------+-----+
|    Bengaluru, India| 3552|
|Greater Mumbai, I...| 3493|
|    Ahmedabad, India| 3491|
|        Delhi, India| 3482|
|    Hyderabad, India|  784|
+--------------------+-----+



#### Gender Distribution

In [22]:
spark.sql('''Select 
            `Gender`,
            count(1) as cnt
          from transactions
          Group by 1
          order by cnt desc
          ''').show()

+------+-----+
|Gender|  cnt|
+------+-----+
|     F|13680|
|     M|12372|
+------+-----+



In [23]:
trxns\
    .groupby('Gender')\
    .count()\
    .sort('count',ascending=False)\
    .show()

+------+-----+
|Gender|count|
+------+-----+
|     F|13680|
|     M|12372|
+------+-----+



#### Card Type used distribution highest to lowest 

In [24]:
spark.sql('''Select 
            `Card Type`,
            count(1) as cnt
          from transactions
          Group by 1
          order by cnt desc
          ''').show()

+---------+----+
|Card Type| cnt|
+---------+----+
|   Silver|6840|
|Signature|6447|
| Platinum|6398|
|     Gold|6367|
+---------+----+



In [25]:
trxns\
    .groupby('Card Type')\
    .count()\
    .sort('count',ascending=False)\
    .show()

+---------+-----+
|Card Type|count|
+---------+-----+
|   Silver| 6840|
|Signature| 6447|
| Platinum| 6398|
|     Gold| 6367|
+---------+-----+



#### Exp Type distribution highest to lowest 

In [26]:
spark.sql('''Select 
            `Exp Type`,
            count(1) as cnt
          from transactions
          Group by 1
          order by cnt desc
          ''').show()

+-------------+----+
|     Exp Type| cnt|
+-------------+----+
|         Food|5463|
|         Fuel|5257|
|        Bills|5078|
|Entertainment|4762|
|      Grocery|4754|
|       Travel| 738|
+-------------+----+



In [27]:
trxns\
    .groupby('Exp Type')\
    .count()\
    .sort('count',ascending=False)\
    .show()

+-------------+-----+
|     Exp Type|count|
+-------------+-----+
|         Food| 5463|
|         Fuel| 5257|
|        Bills| 5078|
|Entertainment| 4762|
|      Grocery| 4754|
|       Travel|  738|
+-------------+-----+



#### Top 5 city and other card type /gender wise / exp type distribution

In [28]:
tf=trxns\
    .groupby('City')\
    .count()\
    .withColumnRenamed('count','ct')\
    .sort('ct',ascending=False)\
    .limit(5)\
    .select('City')\
    .collect()

tf=[ tf[i][0] for i in range(len(tf))]

trxns\
    .filter(trxns.City.isin(tf))\
    .groupby(['City','Gender'])\
    .count()\
    .withColumnRenamed('count','ct')\
    .sort('City')\
    .show()

+--------------------+------+----+
|                City|Gender|  ct|
+--------------------+------+----+
|    Ahmedabad, India|     F|1876|
|    Ahmedabad, India|     M|1615|
|    Bengaluru, India|     M|1592|
|    Bengaluru, India|     F|1960|
|        Delhi, India|     M|1559|
|        Delhi, India|     F|1923|
|Greater Mumbai, I...|     F|1922|
|Greater Mumbai, I...|     M|1571|
|    Hyderabad, India|     F| 389|
|    Hyderabad, India|     M| 395|
+--------------------+------+----+



In [ ]:
from pyspark.sql import functions

In [29]:
spark.sql('''
        Select city,gender ,  count(1) as count
        from transactions 
        where  city in (Select city 
                        from transactions 
                        group by 1 
                        order by count(1) desc 
                        limit 5)             
        group by 1,2 order by 1,2 desc 
          ''').show()

+--------------------+------+-----+
|                city|gender|count|
+--------------------+------+-----+
|    Ahmedabad, India|     M| 1615|
|    Ahmedabad, India|     F| 1876|
|    Bengaluru, India|     M| 1592|
|    Bengaluru, India|     F| 1960|
|        Delhi, India|     M| 1559|
|        Delhi, India|     F| 1923|
|Greater Mumbai, I...|     M| 1571|
|Greater Mumbai, I...|     F| 1922|
|    Hyderabad, India|     M|  395|
|    Hyderabad, India|     F|  389|
+--------------------+------+-----+



In [30]:
spark.sql('''
        Select city,`Card Type` ,  count(1) as count
        from transactions 
        where  city in (Select city 
                        from transactions 
                        group by 1 
                        order by count(1) desc 
                        limit 5)             
        group by 1,2 order by 1,2 desc 
          ''').show()

+--------------------+---------+-----+
|                city|Card Type|count|
+--------------------+---------+-----+
|    Ahmedabad, India|   Silver|  954|
|    Ahmedabad, India|Signature|  828|
|    Ahmedabad, India| Platinum|  900|
|    Ahmedabad, India|     Gold|  809|
|    Bengaluru, India|   Silver|  983|
|    Bengaluru, India|Signature|  859|
|    Bengaluru, India| Platinum|  853|
|    Bengaluru, India|     Gold|  857|
|        Delhi, India|   Silver|  956|
|        Delhi, India|Signature|  872|
|        Delhi, India| Platinum|  791|
|        Delhi, India|     Gold|  863|
|Greater Mumbai, I...|   Silver|  957|
|Greater Mumbai, I...|Signature|  820|
|Greater Mumbai, I...| Platinum|  868|
|Greater Mumbai, I...|     Gold|  848|
|    Hyderabad, India|   Silver|  188|
|    Hyderabad, India|Signature|  192|
|    Hyderabad, India| Platinum|  210|
|    Hyderabad, India|     Gold|  194|
+--------------------+---------+-----+



In [31]:
spark.sql('''
        Select city,`Exp Type` ,  count(1) as count
        from transactions 
        where  city in (Select city 
                        from transactions 
                        group by 1 
                        order by count(1) desc 
                        limit 5)             
        group by 1,2 order by 1,2 desc 
          ''').show(50)

+--------------------+-------------+-----+
|                city|     Exp Type|count|
+--------------------+-------------+-----+
|    Ahmedabad, India|      Grocery|  628|
|    Ahmedabad, India|         Fuel|  760|
|    Ahmedabad, India|         Food|  801|
|    Ahmedabad, India|Entertainment|  622|
|    Ahmedabad, India|        Bills|  680|
|    Bengaluru, India|      Grocery|  608|
|    Bengaluru, India|         Fuel|  780|
|    Bengaluru, India|         Food|  805|
|    Bengaluru, India|Entertainment|  624|
|    Bengaluru, India|        Bills|  735|
|        Delhi, India|      Grocery|  621|
|        Delhi, India|         Fuel|  782|
|        Delhi, India|         Food|  784|
|        Delhi, India|Entertainment|  594|
|        Delhi, India|        Bills|  701|
|Greater Mumbai, I...|      Grocery|  647|
|Greater Mumbai, I...|         Fuel|  762|
|Greater Mumbai, I...|         Food|  742|
|Greater Mumbai, I...|Entertainment|  614|
|Greater Mumbai, I...|        Bills|  728|
|    Hydera

#### Relation of city with amount

In [66]:
spark.sql('''
        Select city,min(Amount) min_Amount, max(Amount) max_Amount , sum(Amount) Total_Amount,avg(Amount) avg_Amount,count(1) as count , skewness(Amount)
        from transactions 
        where  city in (Select city 
                        from transactions 
                        group by 1 
                        order by count(1) desc 
                        limit 5)             
        group by 1 order by 1  
          ''').show(50)

+--------------------+----------+----------+------------+------------------+-----+------------------+
|                city|min_Amount|max_Amount|Total_Amount|        avg_Amount|count|  skewness(Amount)|
+--------------------+----------+----------+------------+------------------+-----+------------------+
|    Ahmedabad, India|      1024|    996291|   567794310| 162645.1761672873| 3491|2.2227071785753116|
|    Bengaluru, India|      1074|    987935|   572326739|161128.02336711713| 3552|2.0961383067370605|
|        Delhi, India|      1005|    996754|   556929212| 159945.2073520965| 3482|2.2995383358307215|
|Greater Mumbai, I...|      1056|    998077|   576751476|165116.36873747496| 3493|2.4142239441853044|
|    Hyderabad, India|      1070|    299751|   114493477| 146037.5982142857|  784|0.0573801105064027|
+--------------------+----------+----------+------------+------------------+-----+------------------+



In [64]:
tf=trxns\
    .groupby('City')\
    .count()\
    .withColumnRenamed('count','ct')\
    .sort('ct',ascending=False)\
    .limit(5)\
    .select('City')\
    .collect()

tf=[ tf[i][0] for i in range(len(tf))]

trxns\
    .filter(trxns.City.isin(tf))\
    .groupby(['City'])\
    .agg(functions.min('Amount').alias("min"),
                              functions.max('Amount').alias("max"),
                              functions.sum('Amount').alias("Total_Amount"), 
                              functions.mean('Amount').alias("avg_Amount"), #functions.avg('Amount')
                              functions.count('Amount').alias("count"),functions.skewness('Amount')\

        )\
    .sort('City')\
    .show()


+--------------------+----+------+------------+------------------+-----+------------------+
|                City| min|   max|Total_Amount|        avg_Amount|count|  skewness(Amount)|
+--------------------+----+------+------------+------------------+-----+------------------+
|    Ahmedabad, India|1024|996291|   567794310| 162645.1761672873| 3491|2.2227071785753116|
|    Bengaluru, India|1074|987935|   572326739|161128.02336711713| 3552|2.0961383067370605|
|        Delhi, India|1005|996754|   556929212| 159945.2073520965| 3482|2.2995383358307215|
|Greater Mumbai, I...|1056|998077|   576751476|165116.36873747496| 3493|2.4142239441853044|
|    Hyderabad, India|1070|299751|   114493477| 146037.5982142857|  784|0.0573801105064027|
+--------------------+----+------+------------+------------------+-----+------------------+



#### Stop Spark

In [67]:
spark.stop()